# Project imports

In [1]:
"""
All needed imports included here
"""
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch
import pytorch_lightning as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Data Loading step

In [2]:
"""
Create data loaders and augmentations needed here
"""
from Data.ShapeNetDataLoader import ShapeNetVoxelData
from Utils.visualization import visualize_occupancy

overfit = False

shapenet_core_path = Path("D:\ShapeNetCoreVoxel32")
shapenet_splits_csv_path = Path("Data/shapenet_splits.csv")
voxel_filename = "model_3.binvox"
# load only models from some synsets
synset_id_filter = ["04379243"]  # tables
train_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="train", 
    overfit=overfit, synset_id_filter=synset_id_filter, voxel_filename=voxel_filename
)
print(f"Train Set Size: {len(train_data)}")
val_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="val",
    overfit=overfit, synset_id_filter=synset_id_filter, voxel_filename=voxel_filename
)
print(f"Validation Set Size: {len(val_data)}")
test_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="test",
    overfit=overfit, synset_id_filter=synset_id_filter, voxel_filename=voxel_filename
)
print(f"Test Set Size: {len(test_data)}")

train_sample = train_data[0]
print(f'Voxel Dimensions: {train_sample.shape}')

visualize_occupancy(train_sample.squeeze(), flip_axes=True)

Train Set Size: 1368
Validation Set Size: 236
Test Set Size: 383
Voxel Dimensions: (1, 32, 32, 32)


Output()

# Reconstruction Networks

# Purifying predicted Meshes

In [70]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

'\nCode to purify meshes predicted by the previous networks to be used in the retrieval step\n'

# Mesh Encoding

In [5]:
#%env CUDA_LAUNCH_BLOCKING=1
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""
from Networks.VoxelAutoencoder import VoxelAutoencoder
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# lower kl_divergence_scale -> smoother latent space but worse reconstruction
kl_divergence_scale=0.05
model = VoxelAutoencoder(train_data, val_data, test_data, device, kl_divergence_scale=kl_divergence_scale)

logger = TensorBoardLogger("tb_logs", name="my_model")
model_checkpoint = ModelCheckpoint(
    monitor="val_loss",
    dirpath="D:/Models/VoxelAutoencoder/",
    filename="voxel-autoencoder-{epoch:0004d}-{val_loss:.4f}",
    save_top_k=3,
    every_n_epochs=8,
    mode="min",
)
tqdm_progess_bar = TQDMProgressBar(refresh_rate=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=64, mode="min")
trainer = pl.Trainer(
    max_epochs=1024,
    gpus=1 if torch.cuda.is_available() else None,
    log_every_n_steps=1,
    logger=logger,
    callbacks=[model_checkpoint, tqdm_progess_bar, early_stopping],
    profiler="simple"
)

trainer.fit(model)

Epoch 1:  81%|████████  | 21/26 [00:23<00:05,  1.10s/it, loss=0.126, v_num=257]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type    | Params
-----------------------------------
0 | _model | Network | 34.2 M
-----------------------------------
34.2 M    Trainable params
0         Non-trainable params
34.2 M    Total params
136.637   Total estimated model params size (MB)


Epoch 334: 100%|██████████| 26/26 [00:07<00:00,  3.66it/s, loss=0.0255, v_num=257]


FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  2514.7         	|  100 %          	|
----------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  7.5035         	|335            	|  2513.7         	|  99.959         	|
run_training_batch                 	|  0.25105        	|7370           	|  1850.2         	|  73.575         	|
optimizer_step_with_closure_0      	|  0.12634        	|7370           	|  931.16         	|  37.029         	|
training_step_and_backward         	|  0.11624        	|7370           	|  856.69         	|  34.067         	|
model_forward                  

In [41]:
# visualize reconstruction
train_test_sample = test_data[1]

visualize_occupancy(train_test_sample.squeeze(), flip_axes=True)

sample_tensor = torch.from_numpy(train_test_sample[np.newaxis, :])

model.eval()
with torch.no_grad():
    decoded_test = model(sample_tensor)
    print(model.encode(sample_tensor)[0].shape)

tmp_decoded = decoded_test.clone()
tmp_decoded[decoded_test<0.5] = 0
tmp_decoded[decoded_test>=0.5] = 1

decoded_test_np = tmp_decoded.squeeze().detach().numpy()

visualize_occupancy(decoded_test_np, flip_axes=True)

Output()

torch.Size([128])


Output()

In [21]:
# compute latent vectors of training samples
latent_vectors = {}
model.eval()
with torch.no_grad():
    for train_sample in train_data:
        sample_tensor = torch.from_numpy(train_sample[np.newaxis, :])
        latent_vectors[model.encode(sample_tensor)[0]] = train_sample

for latent_vector in latent_vectors.items():
    print(latent_vector)
    break

(tensor([-8.1335e-01, -1.5713e+00, -1.0298e+00, -1.7457e+00,  2.8676e-01,
        -8.4046e-01, -3.9098e-02, -1.1492e-02, -1.2321e-01, -3.0048e+00,
        -1.3972e+00,  4.9813e-01,  2.8126e-01, -1.2275e+00,  1.4138e+00,
         1.3568e+00,  2.2159e-01, -1.3605e-01,  1.6306e+00, -6.7599e-01,
         5.0273e-01, -6.4468e-01, -1.1023e+00, -8.7999e-01,  1.7361e+00,
         5.8381e-01, -1.8062e-01,  1.7283e-01,  1.0237e+00, -5.4272e-01,
        -7.5886e-03,  4.3708e-01,  1.4482e+00, -1.4428e+00,  4.6320e-01,
         9.5385e-01, -1.5240e-01, -3.9288e-01, -1.0642e-02, -9.9375e-01,
        -1.5230e-01,  6.6617e-01,  7.0120e-02, -1.3126e+00,  5.2079e-02,
         1.5797e+00, -5.2441e-01, -4.1080e-01, -7.2666e-01, -2.8784e-01,
        -1.1763e+00,  9.6107e-01,  3.9524e-01, -3.0084e-04, -9.0999e-01,
         4.2810e-01,  3.2235e-01, -6.3933e-01, -1.4902e+00, -2.6133e-02,
         1.8495e-01,  6.7956e-01,  7.8874e-01,  4.5258e-01,  1.5660e+00,
        -9.5898e-01, -8.6814e-01,  1.7663e+00, -5.

In [42]:
# compute latent vector of test sample
test_sample = train_data[0]
model.eval()
with torch.no_grad():
    sample_tensor = torch.from_numpy(test_sample[np.newaxis, :])
    test_latent_vector = model.encode(sample_tensor)[0]

print("Test sample:")
visualize_occupancy(test_sample.squeeze(), flip_axes=True)  

# find closest latent vector
min_distance = float('inf')
best_voxel_match = None
for train_latent_vector, train_voxel in latent_vectors.items():
    distance = torch.dist(test_latent_vector, train_latent_vector)
    if (distance < min_distance):
        min_distance = distance
        best_voxel_match = train_voxel
        print(min_distance)

print("Retrieved voxel:")
visualize_occupancy(best_voxel_match.squeeze(), flip_axes=True)   

Test sample:


Output()

tensor(15.6926)
tensor(14.8235)
tensor(14.4839)
tensor(14.3314)
tensor(13.5117)
tensor(13.1566)
Retrieved voxel:


Output()

# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""
# TODO: store latent vector of all shapenet models
# TODO: encode voxel and find obj that has the closest latent vector

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....